# Plot seeds rois from nifti images
This notebook plots the provided seeds on a single image, each seed being specified as a nifti image.

Author: Stephen Larroque
Creation date: 2018-03-18
Version 1.0.0

In [ ]:
%load_ext autoreload
%autoreload 2
# BEWARE: autoreload works on functions and on general code, but NOT on new class methods:
# if you add or change the name of a method, you have to reload the kernel!
# also it will fail if you use super() calls in the classes you change
# ALSO AUTORELOAD SHOULD BE THE FIRST LINE EVER EXECUTED IN YOUR IPYTHON NOTEBOOK!!!

# Profilers:
# http://pynash.org/2013/03/06/timing-and-profiling/
# http://mortada.net/easily-profile-python-code-in-jupyter.html
# use %lprun -m module func(*args, **kwargs)
try:
    %load_ext line_profiler
    %load_ext memory_profiler
except ImportError as exc:
    pass

In [ ]:
# Generate figure inside IPython Notebook (must be called before any import of matplotlib, direct or indirect!)
%matplotlib inline

import math
import matplotlib.pyplot as plt
import matplotlib.colors as pltcol
import numpy as np
import nibabel as nib
from nilearn import image
from nilearn import plotting

In [ ]:
# PARAMETERS - EDIT ME
rois_filepath = ['DMN_Raichle_MPFC_10--1_54_27.nii', 'DMN_Raichle_Cerebellum_R_10-25_-81_-33.nii', 'DMN_Raichle_Cerebellum_L_10--25_-81_-33.nii', 'Boveroux_Thalamus_R_4-4_-12_0.nii', 'Boveroux_Thalamus_L_4--4_-12_0.nii', 'AUD_Maudoux_STG_R_10-44_-6_11.nii', 'AUD_Maudoux_STG_L_10--44_-6_11.nii']  # specify here the filenames to the nifti files of the rois
atlas_cmap = pltcol.ListedColormap(['r', 'b', 'b', 'g', 'g', 'm', 'm'], name='from_list', N=None)  # colormap to use for ROIs, specify the colors in the same order as the rois filenames, or use plt.cm.Set1 if you want to assign random colors
show_center = False # show a black dot at the center of each ROI?
show_colorbar = False  # display the colorbar on the brain image?
dpi_resolution = 300  # dpi resolution when saving on-disk

In [ ]:
def load_maps(list_imgs, voxel_threshold=None):
    if voxel_threshold is None:
        voxel_threshold = 0.0001 # minimum threshold to consider as a voxel and not just background noise (because background voxels can be 0.000001 for example), can be float or str ('1%' to give a percentage). TODO: autodetect minimum value (can be -4, 0.02, etc) as the background and use it as the threshold value.
    # Load masks and resample to first
    imgs = []
    for img in list_imgs:
        im = image.load_img(img)
        if imgs:
            if im.shape != imgs[0].shape:
                im = image.resample_to_img(im, imgs[0])
        im = image.threshold_img(im, voxel_threshold)
        imgs.append(im)
    return imgs

In [ ]:
# Check filepaths are lists and not just string
if isinstance(rois_filepath, str):
    rois_filepath = [rois_filepath]

In [ ]:
def find_cluster_center(im, mricron=False):
    from nilearn.image.resampling import reorder_img, coord_transform
    # Project coordinates to reduced space
    if mricron:
        im2 = reorder_img(im, resample='continuous')  # DO NOT USE: this will convert to MRIcron coordinates space (ie, [0, 100]) but it will mess things up for nilearn!
        # Get indices of nonzero values
        matches = im2.get_data().nonzero()
    else:
        # Get indices of nonzero values
        matches = im.get_data().nonzero()
    # Compute the euclidian middle of the cluster, from the nonzero values indices (= coordinates)
    center = np.mean(matches, axis=1)
    # Project center coordinates to brain space (ie, instead of [0, 100] range, it will be [-50, 50] -- I picked these numbers randomly, you see the idea)
    if not mricron:
        center = coord_transform(center[0], center[1], center[2], im.affine)  # disable this to get MRIcron space
    return center

In [ ]:
# Plot ROIs on glass brain images!
if rois_filepath:
    nb_rois = len(rois_filepath)
    imgs = load_maps(rois_filepath)

    #plotting.plot_prob_atlas(imgs, view_type="filled_contours",
    #                    title="lala", colorbar=True, cut_coords=(0,0,0), draw_cross=True, cmap=pltcol.ListedColormap(['b', 'g', 'r', 'c', 'm'], name='from_list', N=None))
    #plotting.plot_roi(imgs[0])

    centers = []
    fig = plotting.plot_glass_brain(None, cmap=atlas_cmap, alpha=0.5)  # initialize the glass brain images
    for c, im in enumerate(imgs):
        # For each ROI
        # Get the center (to plot the marker)
        center = find_cluster_center(im)
        centers.append(center)
        print('Center found at: ' + str(center))
        # Assign a unique value to this cluster's voxels (to get a different color)
        imdata = im.get_data()  # Convert to numpy structure
        imdata[imdata != 0] = c+1  # Assign unique value
        im2 = nib.Nifti1Image(imdata, affine=im.affine)  # convert back to a nifti file in-memory to supply to nilearn
        # Show colorbar?
        cbar = False
        if show_colorbar and c == (nb_rois-1):
            # Can only plot the colorbar at the last iteration, else nilearn will spit an error (cannot use multiple colorbars)
            cbar = True
        # Plot the clusters
        fig.add_overlay(im2, vmin=1, vmax=nb_rois, cmap=atlas_cmap, colorbar=cbar)
        # Plot the markers (clusters' centers)
        if show_center:
            fig.add_markers([center], marker_color=['k'], marker_size=20)

# Save figure
fig.savefig('rois_glass_brain.png', dpi=dpi_resolution)
print('Image saved in rois_glass_brain.png')